In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from PIL import Image
from numpy import asarray

In [3]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

68616192/68606236 [==============================] - 2s 0us/step
Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [4]:
# Loading base model dari pre-trained model MobileNet V2

IMG_SHAPE = IMG_SIZE + (3,)
MobileNetV2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

MobileNetV2.trainable = False

9420800/9406464 [==============================] - 0s 0us/step


In [5]:
# mendefinisikan modelKu

inputs = tf.keras.Input(shape=(160, 160, 3))
x = tf.keras.layers.Rescaling(1./127.5, offset=-1)(inputs)
x = MobileNetV2(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

modelKu = tf.keras.Model(inputs, outputs)

In [6]:
base_learning_rate = 0.0001
modelKu.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [7]:
# coba performansi model pada validation_dataset sebelum dilakukan proses training

loss0, accuracy0 = modelKu.evaluate(validation_dataset)

32/32 [==============================] - 14s 68ms/step - loss: 0.7657 - accuracy: 0.5370


In [8]:
# Lakukan proses training

initial_epochs = 5
history = modelKu.fit(train_dataset, epochs=initial_epochs, validation_data=validation_dataset)

Epoch 1/5
63/63 [==============================] - 11s 111ms/step - loss: 0.6125 - accuracy: 0.6770 - val_loss: 0.4704 - val_accuracy: 0.8100
Epoch 2/5
63/63 [==============================] - 6s 90ms/step - loss: 0.4051 - accuracy: 0.8560 - val_loss: 0.3320 - val_accuracy: 0.9110
Epoch 3/5
63/63 [==============================] - 6s 89ms/step - loss: 0.2986 - accuracy: 0.9195 - val_loss: 0.2547 - val_accuracy: 0.9440
Epoch 4/5
63/63 [==============================] - 6s 91ms/step - loss: 0.2371 - accuracy: 0.9380 - val_loss: 0.2079 - val_accuracy: 0.9520
Epoch 5/5
63/63 [==============================] - 6s 91ms/step - loss: 0.1981 - accuracy: 0.9480 - val_loss: 0.1765 - val_accuracy: 0.9640


In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(modelKu)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpqbcs5cn8/assets


In [10]:
with open('modelKu.tflite', 'wb') as f:
  f.write(tflite_model)